Iteration 4_INFOSYS 722

In [1]:
# This part starts Data Preparation of Iteration 4
# I will number each relevant cell as answer

In [24]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('iteration4').getOrCreate()

In [25]:
# Import dataset 
dataFrame = spark.read.csv('Dataset/data.csv',header=True,inferSchema=True)

In [26]:
# Show the dataset
dataFrame.show()

+--------------------+--------------------+----------------+----------------------+---------------+-------------------+-------------+---------+
|         ANZSOCGroup|       MapDetailName|          Region|NumberofVictimisations|NumberofRecords|OccurrenceDayOfWeek|       Weapon|YearMonth|
+--------------------+--------------------+----------------+----------------------+---------------+-------------------+-------------+---------+
|AggravatedSexualA...|      HillcrestWest.|   WaikatoRegion|                     1|              1|            UNKNOWN|NotApplicable|  1/07/14|
|Non-AggravatedSex...|   TeAwamutuCentral.|   WaikatoRegion|                     1|              1|                Sun|NotApplicable|  1/07/14|
|AggravatedSexualA...|      TotaraHeights.|  AucklandRegion|                     1|              1|                Tue|    NotStated|  1/07/14|
|AggravatedSexualA...|       FlaxmereEast.|Hawke'sBayRegion|                     1|              1|                Sun|    NotStated|  1

In [27]:
# 2.3 Data Exploration
print(dataFrame.head(2))

dataFrame.describe().show()

dataFrame.select('ANZSOCGroup','Region','YearMonth').describe().show()

[Row(ANZSOCGroup='AggravatedSexualAssault', MapDetailName='HillcrestWest.', Region='WaikatoRegion', NumberofVictimisations=1, NumberofRecords=1, OccurrenceDayOfWeek='UNKNOWN', Weapon='NotApplicable', YearMonth='1/07/14'), Row(ANZSOCGroup='Non-AggravatedSexualAssault', MapDetailName='TeAwamutuCentral.', Region='WaikatoRegion', NumberofVictimisations=1, NumberofRecords=1, OccurrenceDayOfWeek='Sun', Weapon='NotApplicable', YearMonth='1/07/14')]
+-------+--------------------+-------------+---------------+----------------------+---------------+-------------------+-----------+---------+
|summary|         ANZSOCGroup|MapDetailName|         Region|NumberofVictimisations|NumberofRecords|OccurrenceDayOfWeek|     Weapon|YearMonth|
+-------+--------------------+-------------+---------------+----------------------+---------------+-------------------+-----------+---------+
|  count|                8354|         7734|           7734|                  8354|           8354|               8354|       83

In [20]:
# 3.2 To clean the data, issues must be made explicit, then explicitly resolved.
# Show orginal columns before cleaning
print("Total data columns before cleaning:", dataFrame.count())

Total data columns before cleaning: 8354


In [18]:
# Drop empty rows
droppedTable = dataFrame.na.drop()

In [19]:
# Show columns after cleaning
print("Total data columns after cleaning:", droppedTable.count())


Total data columns after cleaning: 7734


In [22]:
#
droppedTable.groupBy('ANZSOCGroup').mean().show()

AttributeError: 'GroupedData' object has no attribute 'show'